In [ ]:
def create_rsa_key(rsa_length_bits):
    len_bytes = Integer(rsa_length_bits//2)
        
    e = 65537
    p = random_prime(2**(len_bytes), lbound = 2**(len_bytes - 1), proof=True)
    q = random_prime(2**(len_bytes), lbound = 2**(len_bytes - 1), proof=True)
    n = Integer(p*q)
    
    phi_n = Integer((p - 1)*(q - 1))
    
    _, d,_ = xgcd(e, phi_n)
    d = d if d >= 0 else d + phi_n

    d = Integer(floor(d/(p-1))*(p-1) + d % (p - 1)) # d = floor(d/(p-1))*(p-1) + d mod (p - 1) -> Euclidean division
    
    d_p = mod(d, p - 1) # d_p = d mod (p - 1)
    d_q = mod(d, q - 1) # d_q = d mod (q - 1)
    q_inv = power_mod(q, -1, p) # q_inv = q^(-1) mod p
    
    return n, e, p, q, d, d_p, d_q, q_inv

def sign_message(message, hash_func, p, q, d_p, d_q, q_inv, rsa_length_bits):
    #calculates sigma and converts it to bytes in bigendian byteorder
    
    e = 65537
    n = Integer(p*q)
    phi_n = (p - 1)*(q - 1)
    _, d,_ = xgcd(e, phi_n)
    
    d = d if d >= 0 else d + phi_n

    # === HASH IT ===
    m = hash_func(message)
    m = int(m.hexdigest(), 16)
    
    # === START USED ===
    
    m_1 = power_mod(m, d_p, p)
    m_2 = power_mod(m, d_q, q)
    h = q_inv * (m_1 - m_2) % p
    sigma = m_2 + h * q
    
    # === END USED ===
    
    bytes_len = Integer(rsa_length_bits // 8)
    sigma_bytes = int.to_bytes(int(sigma), bytes_len, 'big')
    
    return sigma_bytes
    
def is_signature_ok(message, hash_func, signature, n, e):
    sig_to_int = int.from_bytes(signature, byteorder = 'big')
                              
    m = hash_func(message)
    mess_to_int = int(m.hexdigest(), 16)  
        
    sigma = power_mod(sig_to_int, e, n)
                  
    return mess_to_int == sigma
    
import unittest
import hashlib

class Digital_signature_with_RSA_tests(unittest.TestCase):
    def __init__(self, *args, **kw_args):
        super(Digital_signature_with_RSA_tests, self).__init__(*args, **kw_args)
        
        self.rsa_length_bits = 512
        self.message = b'egy aprocska kalapocska, benne csacska macska mocska'
        self.hash_func = hashlib.sha256
        self.n = 5729663077757322405724886857654621604006775110813919951261130149520327478307779221868640435713466100413220954390777583730910781794709934827360388197456627
        self.e = 65537
        self.p = 93823806108084397377997416420175991891198870731721791140478154122834102438129
        self.q = 61068329195223532562492566114152810373313790406005256685696432844784551510563
        self.d = 2622267030139510936376585080313299668745582078425205838200197410694915885444235838473255013362513803794162299890166792097086729695866937831306601304423105
        self.d_p = 1186809516206142567196543299393104616900435842907019925468916638964698886449
        self.d_q = 45343958913254612532706917500753834603143196495674623507932310223124431170285
        self.q_inv = 53925405957980236628208181039671942739794007092753729429983308049670277790924
        self.signature = b'L_bMj\xd9I\x93b\xfaC\xbc\xb0\x8fF\xac\x0fY\x8c\xe40>EoW\x94\xf2pD\x9by\x944\x1d\x85&,\xb9\xb4C\x06q\x14\x83\xd5\xfc\x18\xa2?\xe6\xee3\xca\xe0\xb8"ea\xd9>\xfc+\xf0+'

    def test_create_rsa_key(self):
        rsa_length_bits = 256
        n, e, p, q, d, _, _, q_inv = create_rsa_key(rsa_length_bits)
        
        self.assertEqual(n, p * q)
        
        bits_required = floor(log(n, 2)) + 1
        self.assertLessEqual(bits_required, rsa_length_bits)
        self.assertGreaterEqual(bits_required, rsa_length_bits - 1)
        
        phi_n = (p - 1) * (q - 1)
        self.assertEqual(e * d % phi_n, 1)
        self.assertGreater(e, 0)
        self.assertGreater(d, 0)
        self.assertLess(e, phi_n)
        self.assertLess(d, phi_n)
        
        self.assertEqual(q * q_inv % p, 1)
        self.assertGreater(q_inv, 0)
        self.assertLess(q_inv, p)

    def test_sign_message(self):
        signature = sign_message(
            self.message,
            self.hash_func,
            self.p, self.q, self.d_p, self.d_q, self.q_inv,
            self.rsa_length_bits
        )
        
        self.assertEqual(signature, self.signature)

    def test_is_signature_ok(self):
        self.assertTrue(is_signature_ok(self.message, self.hash_func, self.signature, self.n, self.e))
        self.assertFalse(is_signature_ok(b'', self.hash_func, self.signature, self.n, self.e))

test_runner = unittest.TextTestRunner(stream=sys.stdout, verbosity=2)
test_runner.run(unittest.defaultTestLoader.loadTestsFromTestCase(Digital_signature_with_RSA_tests))